# Извлекаем из данных соревнования Raiffeisen Data Cup информацию по банкоматам и карточным терминалам
Источник данных: https://boosters.pro/champ_11

Данные могли быть заведомо зашумлены, но попытка - не пытка

In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


downloaded = drive.CreateFile({'id': '1Plpet7kjTUQZvdKSV9bZUE1w6yXfEuQO'})
print('Downloaded content "{}"'.format((downloaded['title'])))
downloaded.GetContentFile(downloaded['title'])

downloaded = drive.CreateFile({'id': '1mtLiIBIanehQ46CkPkf9gtFQcCGBxk2Z'})
print('Downloaded content "{}"'.format((downloaded['title'])))
downloaded.GetContentFile(downloaded['title'])

downloaded = drive.CreateFile({'id': '1GH_3M-upAPAK0wEyihvWXSMpePqu8MQN'})
print('Downloaded content "{}"'.format((downloaded['title'])))
downloaded.GetContentFile(downloaded['title'])

Downloaded content "test_set.csv"
Downloaded content "train_set.csv"
Downloaded content "tagged_nodes.pickle"


In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv('train_set.csv', usecols = ['pos_adress_lat', 'pos_adress_lon', 'atm_address_lat', 'atm_address_lon','terminal_id'])
train.rename(columns = {'pos_adress_lat': 'pos_address_lat', 'pos_adress_lon': 'pos_address_lon'}, inplace = True)

test = pd.read_csv('test_set.csv', usecols = ['pos_address_lat', 'pos_address_lon', 'atm_address_lat', 'atm_address_lon'])

In [0]:
df = train.append(test)

In [5]:
df.sample(3)

,atm_address_lat,atm_address_lon,pos_address_lat,pos_address_lon,terminal_id
423100,NaN,NaN,60.707443,28.742137,7db6fdbfcdecc3c57f6cf9b1317aeea0
759739,NaN,NaN,55.998782,92.922122,NaN
791530,NaN,NaN,52.584410,39.523001,NaN


In [6]:
df_pos = df[['pos_address_lat','pos_address_lon','terminal_id']]
df_atm = df[['atm_address_lat','atm_address_lon','terminal_id']]

df_pos.rename(columns = {'pos_address_lat': 'lat', 'pos_address_lon': 'lon'}, inplace = True)
df_atm.rename(columns = {'atm_address_lat': 'lat', 'atm_address_lon': 'lon'}, inplace = True)

df_pos.dropna(inplace=True)
df_atm.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df_pos.shape, df_atm.shape

((942614, 3), (184680, 3))

In [0]:
# ограничиваем квадратом, важным для соревнования

LAT_MIN, LAT_MAX = 55.309397, 56.13526
LON_MIN, LON_MAX = 36.770379, 38.19270

In [0]:
# группируем по терминалам

df_pos = df_pos.groupby('terminal_id').mean()
df_atm = df_atm.groupby('terminal_id').mean()

In [10]:
df_pos = df_pos[df_pos['lat'] >= LAT_MIN]
df_pos = df_pos[df_pos['lat'] <= LAT_MAX]
df_pos = df_pos[df_pos['lon'] >= LON_MIN]
df_pos = df_pos[df_pos['lon'] <= LON_MAX]


df_pos.shape

(67987, 2)

In [11]:
df_pos.sample(3)

,lat,lon
terminal_id,,
9b9ab3bdee935a818f42685f94c7ec26,55.749879,37.878682
fd9a2b521cd5daafc5a767d9f8dcc84c,55.431991,37.547381
66d61ecc1a744b104b1c75170b2e68e8,55.766357,37.610736


In [12]:
df_atm = df_atm[df_atm['lat'] >= LAT_MIN]
df_atm = df_atm[df_atm['lat'] <= LAT_MAX]
df_atm = df_atm[df_atm['lon'] >= LON_MIN]
df_atm = df_atm[df_atm['lon'] <= LON_MAX]


df_atm.shape

(2277, 2)

## Отобразим на карте
(чтобы не тормозило, показана 1 точка из 100, выбранная случайным образом)

In [13]:
!pip install folium

In [14]:
import random

import folium

fmap = folium.Map([55.753722, 37.620657])

for index, row in df_atm.iterrows():
  if random.randint(1, 22) == 3:
    folium.Marker(
        location=[row.lat, row.lon],
        icon=folium.Icon(color='black'),
        popup='ATM'
    ).add_to(fmap)

for index, row in df_pos.iterrows():
  if random.randint(1, 680) == 3:
    folium.Marker(
        location=[row.lat, row.lon],
        icon=folium.Icon(color='orange'),
        popup='POS'
    ).add_to(fmap)
    
fmap.save('map_demo.html')

fmap

Итого, у нас **67987** торговых точек с pos-терминалами и **2277** банкоматов

## Сравним с тем, что мы можем получить из данных OpenStreetMap

In [0]:
# используется готовый объект с нодами, сгенерированный из данных OSM в официальном бейслайне 
# https://github.com/datasouls/mts-geohack/blob/master/Geohack112_StarterKit.ipynb
# и подсчет тегов из https://github.com/DenisVorotyntsev/MTCGeohack.112/blob/master/starter.ipynb

tagged_nodes = pd.read_pickle('tagged_nodes.pickle')

osm_tags = []
for i in range(len(tagged_nodes)):
    if (len(tagged_nodes[i].tags)>0):
        for j in tagged_nodes[i].tags:
            osm_tags.append(j)
tags, count = np.unique(osm_tags,return_counts=True)

tags_count = pd.DataFrame({'tag':tags, 
                           'count':count}).sort_values(by='count',ascending=False).reset_index(drop=True)

In [16]:
tags_count[tags_count['tag']=='atm']

,count,tag
88,1078,atm


In [17]:
tags_count[tags_count['tag']=='shop']

,count,tag
5,36349,shop


Итого, на OSM отмечено **36349** shop **1078** atm то есть примерно в два раза меньше объектов, чем в датасете банка и можно поробовать получить из датасета банка выгоду

In [0]:
# сохраняем
df_pos.to_csv('raif_pos.csv', index=False)
df_atm.to_csv('raif_atm.csv', index=False)

from google.colab import files

files.download('raif_pos.csv')
files.download('raif_atm.csv')

In [0]:
from google.colab import files
files.download('map_demo.html')